<a href="https://colab.research.google.com/github/nat1060/DeepQ-L/blob/main/Partial_Code_MyDeep_Q_Learning_for_Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.5 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=d6d0a22b81e3c74bbab7788b437fad232867707bca773f822da11aac5c8cb1df
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be install

### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

In [3]:
class NeuralNetwork(nn.Module):

  #Defining the Neural Network Object
  def __init__(self, state_size, action_size, seed = 42):
      super(NeuralNetwork, self).__init__()
      self.seed = torch.manual_seed(seed)
      #Adding the Layers
      self.fc1 = nn.Linear(state_size, 64)
      self.fc2 = nn.Linear(64, 64)
      self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x) #What's This funtion ?
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

### Creating the architecture of the Neural Network

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ',number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99 # Gamma, PV of future rewards, low is short sided only looking at current rewards, or high, mainly looking into the future
replay_buffer_size = int(1e5)  # size of the memory of AI, stabilize and improve training process, 100K experiences in AI memory
interpolation_parameter = 1e-3  #Tau, used in the subupdate of the target network

### Implementing Experience Replay

In [6]:
class ReplayMemory(object):

  def __init__(self, capacity):
     self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
     self.capacity = capacity
     self.memory = [] # empty list that stores the experiences

# Adds an Experience event into the memory buffer
  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity: #delete the first experience in the list if the list gets to the limit
      del self.memory[0]

# Samples randomly select a batch of experiences from the memory buffer
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    # A for loop iterating through the experiences starting at the first element 0; then convert to Torch Pytorch tensor?, datatype is float, then move to designated device, GPU or CPU
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device) # has to be cast as a uint8 before float
    return states, next_states, actions, rewards, dones


### Implementing the DQN class

In [10]:
class Agent():
  # Created the agent class that define the behavior of an agent that interact that uses a Deep Q network (using local_qnetwork(select the actions ) and  the
  # target_qnetwork(calculate the target Q values that will be used in the training of the local_qnetwork )) and they interact
  # This double Q network will stabilize this learning process; then updating the target_qnetwork parameters by blending them with those of the local_qnetwork to prevent abrupt changes that could de-stablizize the training
  # Implement Act method, helps the Agent choose an action based on its current understanding of the optimal policy; those actions will be returned from the local_qnetwork that will forward propagate the state to return the action values
  # ... and then following an eval policy will return the final action. The epsilon greedy? strategy is used as an expiration mechanism; The random method allows the agent to learn with more options
  # The learn method uses experiences that are sampled from the Replay memory next states, the local_qnetworks q values towards the target_qnetwork values
  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size #also number_actions
    self.local_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
    self.target_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate) #Implements Adam algorithm
    self.memory = ReplayMemory(replay_buffer_size)
    self.time_step = 0

  def step(self, state, action, reward, next_state, done): # state, action, reward, next_state, done
    self.memory.push((state, action, reward, next_state, done)) #Stores experience and decides when to learn from the experience
    self.time_step = (self.time_step + 1) % 4 # Increment; Reset every 4 steps
    if self.time_step == 0: #We will learn now
      if len(self.memory.memory) > minibatch_size: #Instance of the ReplyMemory class referencing 'memory' attribute
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor) #Passing in gamma

  def act(self, state, epsilon = 0.):  #state is a numpy array and needs to be a torch sensor
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #Which batch this states belongs to
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state) #Inference Mode; not training mode
    self.local_qnetwork.train() # Back to training mode
    #If randmom generated number is larger than epsilon, then we select the action with the highest Q Value, otherwise if it's less, we just select random action
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy()) #Send to CPU; expects a numpy data format
    else:
      return random.choice(np.arange(self.action_size)) #Uses a random choice that takes a value from our action_sizes, 0,1,2,3

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences # The tuples of these elements
    # Action values of our target qNetowrk propagating the Next States; We want the Maximum of these Action Values of the next states
    #We detach the resulting tensor from the computation graph, means we won't be tracking gradients for this tensor during the backward propagation
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1) # We need dimension 1 and then use brackets to select q values tensor; then add the dimension of the batch in the unsqueeze
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)  #REMEMBER THIS FORMULA; WE'LL USE IT OFTEN
    q_expected = self.local_qnetwork(states).gather(1, actions)
    # Calculate the Loss
    loss = F.mse_loss(q_expected, q_targets)
    #zero reset
    self.optimizer.zero_grad()
    #back propagate and zero reset
    loss.backward()
    #Updates the parameters of the model
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    #We obtain the parameters for the NN Model we inherited from
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()): #What is zip() ?
      #Updating using the weighting of the target and local parameters
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)



### Initializing the DQN agent

In [11]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [13]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  #Dynamic Print - keeping only the average rewards over 100 actions
  print('\r Episode {}\t Average Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "") #end provides the dynamic overwriting effect
  if episode % 100 == 0: #Resetting every 100
    print('\r Episode {}\t Average Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\n Environment solved in {:d} episodes! \t Average Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break




 Episode 100	 Average Score: -83.29
 Episode 200	 Average Score: -14.94
 Episode 300	 Average Score: 65.96
 Episode 400	 Average Score: 162.55
 Episode 493	 Average Score: 200.53
 Environment solved in 393 episodes! 	 Average Score: 200.53


## Part 3 - Visualizing the results

In [14]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()